 # Подготовка Google Colabaratory к работе с CUDA и C 



In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin
!git clone https://github.com/NVIDIA/cuda-samples/
!cp cuda-samples/Common/* /usr/local/include
!nvcc --version

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-ok2h0u_6
  Running command git clone -q https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-ok2h0u_6
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4306 sha256=691fa5616ed486f632420efb78654b91c01683ea932ca8b3d12bc08d7993802f
  Stored in directory: /tmp/pip-ephem-wheel-cache-m_6281mn/wheels/ca/33/8d/3c86eb85e97d2b6169d95c6e8f2c297fdec60db6e84cb56f5e
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out
Cloning into 'cuda-samples'...
remote: Enumerating objects: 9693, done.
remote: Total 9693 (delta 0), reused 0 (delta 0), pack-reused 9693
Receiving objects: 100% (9693/9693), 122.53 MiB | 26.63 MiB/s, done.
Resolving deltas: 100% (7840/7840), done.
Checking out files: 100% (3612/3612), done.
cp: -r not specified; omi

# **Алгоритм для CPU**


In [ ]:
%%cu

#include <iostream>
#include <cmath>
#include <fstream>
#include <ctime>

using namespace std;
void go_to_bins(int n, double* signal);
double find_premax(double* signal, int n);

double* ak_func_demo(double* signal, int n) {
    double* acf = new double[n];
    for (int i = 0; i < n; i++) {
        int k = 0;
        acf[i] = 0.0;
        for (int j = 0; j < n; j++) {
            if (i + j < n) {
                k++;
                acf[i] += signal[i + j] * signal[j];

            }
        }
    }
    return acf;
    delete[] acf;
}


int main() {
    int time = clock(), times = time;
    times = clock() - time;
    ofstream fw("calc_results1.txt", std::ofstream::out);

    //Maximum signal length
    int end = 14;

    //Перебор длин сигналов
    for (int j = 5; j < end; j++) {
        
        //min_acf[0] - сигнал, min_acf[1] - значение АКФ
        int min_acf[2] = { 0, 9999999 };
        
        int size = j;
        double* signal = new double[size];
        int now_acf;
        int l1 = pow(2, size);
        int l0 = pow(2, size - 1);
        for (int i = l0; i < l1; i++) {
            go_to_bins(i, signal);

            double* acf = ak_func_demo(signal, size);
            now_acf = find_premax(acf, size);
            if (min_acf[1] >= now_acf) {
                min_acf[1] = now_acf;
                min_acf[0] = i;
            }
            delete[] acf;
        }


        times = clock() - time;


        fw << "signal = " << min_acf[0] << " size = " << size << " acf = " << min_acf[1] << " time = " << times << "ms\n";
        delete[] signal;
    }
    
    
    
}

void go_to_bins(int n, double* signal) {
    int size = int(log2(n)) + 1;
    for (int i = 0; i < size; i++) {
        signal[i] = n >> i & 1;
        if (signal[i] == 0)
            signal[i] = -1;
    }
}


double find_premax(double* signal, int n) {
    int temp = 0;
    int curmax = 0;
    for (int i = 0; i < n; i++) {
        if (abs(signal[i]) >= curmax) {
            temp = curmax;
            curmax = abs(signal[i]);
        }
        if (abs(signal[i]) > temp && abs(signal[i]) < curmax) temp = abs(signal[i]);
    }
    return temp;
}


# **Реализация алгоритма для GPU**

**Версия для нахождения наилучшего сигнала длины N. N = const (демонстрационная)**

In [ ]:
%%cu
//Версия для проверки сигнала одной длины 
#include <cstdio>
#include <cinttypes>
#include <chrono>
#include <bitset>


#define SIGNAL_LENGTH_MIN 5
#define SIGNAL_LENGTH_MAX 64


__global__ void calculate_acf(uint64_t start_offset, uint64_t end, uint64_t *min_result_sidelobe_amp, uint64_t *result_signal, uint64_t n) {
    
    // Инициализация переменных
    __shared__ int8_t acf_container[SIGNAL_LENGTH_MAX];
    __shared__ uint8_t signal_binary[SIGNAL_LENGTH_MAX];
    __shared__ uint64_t result_sidelobe_amp;

    // Сдвиг по сигналу
    size_t idx = threadIdx.x; 

    // Сигнал в десятичной системе счисления
    uint64_t signal_decimal = blockIdx.x + start_offset; 

    while (signal_decimal <= end) {
        
        // Сброс амплитуды и разбиение сигнала на биты
        result_sidelobe_amp = 0;
        signal_binary[n - idx - 1] = (signal_decimal >> idx) & 1; //0 -> -1
        __syncthreads();

        // Начала расчета АКФ для каждого сдвига
        int8_t temp = 0;
   
        //Превращаем сигнал из вида {0;1} в {1;1} и расчитываем АКФ    
        for (size_t i = 0; i < n - idx; i++)  temp += (signal_binary[i + idx]*2-1) * (signal_binary[i]*2-1);
        
  
        // Взятие АКФ по модулю 
		    if(temp<0) temp*=(-1);
        acf_container[idx] = temp;

        //вычисление амплитуды бокового лепестка
        if (idx != 0) atomicMax(reinterpret_cast<unsigned long long int*>(&result_sidelobe_amp), (unsigned long long)acf_container[idx]);
        __syncthreads();

        // Проверяем, лучшие ли имеющаяся амплитуда бокового листка
        if (idx == 0) {
          uint64_t old = atomicMin(reinterpret_cast<unsigned long long int*>(min_result_sidelobe_amp), (unsigned long long)result_sidelobe_amp);
          if (old >= result_sidelobe_amp) {
              *result_signal = signal_decimal;
          }
        }
        signal_decimal += gridDim.x;
        __syncthreads();
    }
}


int is_goodlen(int n){
    if(n<SIGNAL_LENGTH_MIN || n>=SIGNAL_LENGTH_MAX){ 
        printf("Wrong signal length!"); 
        return 0;
    } 
    return 1;
}


uint64_t get_start_byblen(int n) {
	return 1ULL << (n - 1);
}


uint64_t get_end_byblen(int n) {
	return (1ULL << n) - 1ULL;
}


int main() {
    
    //длительность сигнала. 
    int n = 15;

    //Проверка на длину сигнала
    if(!is_goodlen(n)) return -1;
  
    //Отсчет времени пошёл
    auto start_time = std::chrono::high_resolution_clock::now();

    //Инициализация переменных
    uint64_t *gpu_temporary_sidelobe_amp;
    uint64_t *gpu_temporary_result_signal;
    uint64_t end = get_end_byblen(n);
    uint64_t start = get_start_byblen(n);
    uint64_t result_sidelobe_amp;
    uint64_t result_signal;

    //Выделяем память
    cudaMalloc((void**)&gpu_temporary_sidelobe_amp, sizeof(uint64_t));
    cudaMalloc((void**)&gpu_temporary_result_signal, sizeof(uint64_t));
    cudaMemset(gpu_temporary_sidelobe_amp, 0xFF, sizeof(uint64_t));

    //Расчитываем наилучшую АКФ для сигнала длины N   
    calculate_acf<<<3072, n>>>(start, end, gpu_temporary_sidelobe_amp, gpu_temporary_result_signal, n);

    //Передаем данные с GPU на CPU для дальнейшей работы
    cudaMemcpy(&result_sidelobe_amp, gpu_temporary_sidelobe_amp, sizeof(uint64_t), cudaMemcpyDeviceToHost);
    cudaMemcpy(&result_signal, gpu_temporary_result_signal, sizeof(uint64_t), cudaMemcpyDeviceToHost);

    //Подготовка к выводу и сам вывод
    std::bitset<SIGNAL_LENGTH_MAX> s(result_signal);
    printf("Best signal is %s (%llu) with result_sidelobe_amp of %llu\n", s.to_string().c_str(), result_signal, result_sidelobe_amp);

    //Расчет времени окончен
    auto end_time = std::chrono::high_resolution_clock::now();
    //Время указано в микросекундах
    printf("Calculation took %f nanoseconds\n", (double)(std::chrono::duration_cast<std::chrono::microseconds>(end_time - start_time).count()) );
    return 0;
}

**Версия для  перебора от 5 до 63 с сохранением данных в файл**

In [ ]:
%%cu

#include <cstdio>
#include <cinttypes>
#include <chrono>
#include <fstream>
#include <bitset>
using namespace std;

#define SIGNAL_LENGTH_MIN 5
#define SIGNAL_LENGTH_MAX 64
#define TM_SPACES "                                        "


__global__ void calculate_acf(uint64_t start_offset, uint64_t end, uint64_t *min_result_sidelobe_amp, uint64_t *result_signal, uint64_t n) {
    
    // Инициализация переменных
    __shared__ int8_t acf_container[SIGNAL_LENGTH_MAX];
    __shared__ uint8_t signal_binary[SIGNAL_LENGTH_MAX];
    __shared__ uint64_t result_sidelobe_amp;

    // Сдвиг по сигналу
    size_t idx = threadIdx.x; 

    // Сигнал в десятичной системе счисления
    uint64_t signal_decimal = blockIdx.x + start_offset; 

    while (signal_decimal <= end) {
        
        // Сброс амплитуды и разбиение сигнала на биты
        result_sidelobe_amp = 0;
        signal_binary[n - idx - 1] = (signal_decimal >> idx) & 1;
        __syncthreads();

        // Начала расчета АКФ для каждого сдвига
        int8_t temp = 0;

        //Превращаем сигнал из {0;1} в {-1;1} и расчитываем АКФ
        for (size_t i = 0; i < n - idx; i++)  temp += (signal_binary[i + idx] ^ signal_binary[i]) * 2 - 1;
        
  
        // Взятие АКФ по модулю 
		    if(temp<0) temp*=(-1);
        acf_container[idx] = temp;

        //вычисление амплитуды бокового лепестка
        if (idx != 0) atomicMax(reinterpret_cast<unsigned long long int*>(&result_sidelobe_amp), (unsigned long long)acf_container[idx]);
        __syncthreads();

        // Проверяем, лучшие ли имеющаяся амплитуда бокового листка
        if (idx == 0) {
          uint64_t old = atomicMin(reinterpret_cast<unsigned long long int*>(min_result_sidelobe_amp), (unsigned long long)result_sidelobe_amp);
          if (old >= result_sidelobe_amp) {
              *result_signal = signal_decimal;
          }
        }
        signal_decimal += gridDim.x;
        __syncthreads();
    }
}


int is_goodlen(int n){
    if(n<SIGNAL_LENGTH_MIN || n>=SIGNAL_LENGTH_MAX){ 
        printf("Wrong signal length!"); 
        return 0;
    } 
    return 1;
}


uint64_t get_start_byblen(int n) {
	return 1ULL << (n - 1);
}


uint64_t get_end_byblen(int n) {
	return (1ULL << n) - 1ULL;
}

int main() {
    
    //Создание и первичное оформление файла
    ofstream result_file;
    result_file.open("signal_acf_calculation_results.dat",  std::ios_base::app);
    result_file<<"Signal Length"<<TM_SPACES<<"Signal"<<TM_SPACES<<"Signal Dec"<<TM_SPACES<<"ACF"<<TM_SPACES<<"Calculation time (mks)"<<'\n';

    //длительность сигнала. 
    int n;
    result_file.close();
    for(n=SIGNAL_LENGTH_MIN; n<SIGNAL_LENGTH_MAX; n++){
      result_file.open("signal_acf_calculation_results.dat",  std::ios_base::app);
      auto start_time = std::chrono::high_resolution_clock::now();
       if(n<10) result_file<<"       ";
       else result_file<<"      ";
       result_file<<n<<"               ";

        //Инициализация переменных
        uint64_t *gpu_temporary_sidelobe_amp;
        uint64_t *gpu_temporary_result_signal;
        uint64_t end = get_end_byblen(n);
        uint64_t start = get_start_byblen(n);
        uint64_t result_sidelobe_amp;
        uint64_t result_signal;

        //Выделяем память
        cudaMalloc((void**)&gpu_temporary_sidelobe_amp, sizeof(uint64_t));
        cudaMalloc((void**)&gpu_temporary_result_signal, sizeof(uint64_t));
        cudaMemset(gpu_temporary_sidelobe_amp, 0xFF, sizeof(uint64_t));

        //Расчитываем наилучшую АКФ для сигнала длины N   
        calculate_acf<<<3071, n>>>(start, end, gpu_temporary_sidelobe_amp, gpu_temporary_result_signal, n);

        //Передаем данные с GPU на CPU для дальнейшей работы
        cudaMemcpy(&result_sidelobe_amp, gpu_temporary_sidelobe_amp, sizeof(uint64_t), cudaMemcpyDeviceToHost);
        cudaMemcpy(&result_signal, gpu_temporary_result_signal, sizeof(uint64_t), cudaMemcpyDeviceToHost);

        //Подготовка к выводу и сам вывод
        std::bitset<SIGNAL_LENGTH_MAX> s(result_signal);
        auto end_time = std::chrono::high_resolution_clock::now();
        result_file<<s.to_string().c_str()<<"               "<<result_signal<<"                                             "<<result_sidelobe_amp<<"                                                  "<<(double)(std::chrono::duration_cast<std::chrono::microseconds>(end_time - start_time).count())<<'\n';
        result_file.close();
    }
    
    printf("Calculations done!");
    return 0;
}